In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [ ]:
# read csv with date columns formated as datetime64
df = pd.read_csv("data/Crime_Data_from_2010_to_Present.csv", parse_dates=['Date Reported', 'Date Occurred'], low_memory=True)
df.columns

In [ ]:
df.columns=[x.strip().replace(" ","") for x in df.columns]

### Creating a sub data frame called df_clean that only contains the columns of interests before cleaning the data

In [ ]:
df_clean=df[['DRNumber','DateOccurred', 'TimeOccurred', 'AreaID',
       'AreaName', 'CrimeCode', 'CrimeCodeDescription',
        'VictimAge', 'VictimSex', 'VictimDescent',
        'Location']]
df_clean.columns

In [ ]:
df_clean.count()

### Deleting all rows with NaN values

In [ ]:
df_clean=df_clean.loc[(~(df["DRNumber"].isna())) &
                (~(df["DateOccurred"].isna())) &
                (~(df["TimeOccurred"].isna())) &
                (~(df["AreaID"].isna())) &
                (~(df["AreaName"].isna())) &
                (~(df["CrimeCode"].isna())) &
                (~(df["CrimeCodeDescription"].isna())) &
                (~(df["VictimAge"].isna())) &
                (~(df["VictimSex"].isna()))&
                (~(df["VictimDescent"].isna())) &
                (~(df["Location"].isna())) ]

In [ ]:
df_clean.count()

In [ ]:
df_clean.dtypes

In [ ]:
df_clean.to_csv("df_clean.csv",index= False)

In [ ]:
crime_cat_df= pd.read_excel("crimecode_desc_update.xlsx")
crime_cat_df.head()

In [ ]:
cc_df = df_clean.merge(crime_cat_df, on="CrimeCodeDescription", how="outer")

In [ ]:
cc_df["NewCrimeCategory"].value_counts()

In [ ]:
cc_df.shape

In [ ]:
# Separate Location Column into separate lat_lng coordinates and create zipcode list for 60,926 unique Location coords

from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)

df_coords = cc_df['Location'].unique()
zip_codes = []
location_list = []
pop_list = []

#Print completed for every 1000

for i, coord in enumerate(df_coords):
    if i % 1000 == 0: print('Completed', i)
    location_list.append(coord)
    coord = coord.replace(' ', '')[1:-1]
    splits = coord.split(',')
    lat = float(splits[0])
    long = float(splits[1])
    zipcodes = search.by_coordinates(lat=lat, lng=long)

    if zipcodes:
        zip_codes.append(zipcodes[0].zipcode)
        pop_list.append(zipcodes[0].population)
    else:
        zip_codes.append('not found')
        pop_list.append(0)

In [ ]:
len(cc_df['Location'].unique())

In [ ]:
# create new zipcode_df to merge with cc_df 

zipcode_df = pd.DataFrame({'Location': location_list, 'Zip_codes': zip_codes, 'Population': pop_list})
zipcode_df.head()

In [ ]:
zipcode_df.info()

In [ ]:
#save new zipcode_df as csv file for merge

zipcode_df.to_csv("zip_codes.csv",index = False)

In [ ]:
#merge the dataframes based on Lacation in order to add 2 new columns 'Zip_codes' and 'Population' to 

zc_df = cc_df.merge(zipcode_df, on="Location", how="outer")

In [ ]:
zc_df.head()

In [ ]:
zc_df.shape

In [ ]:
zc_df.nunique()

In [ ]:
#save final merged clean_df to new csv file to use for Data Analysis and plotting
zc_df.to_csv("df_clean2.csv",index = False)

In [ ]:
#display count of Columns to make sure all have same amount of rows
zc_df.count()

**Please add new code below this cell ONLY